# QuantETF Backtest Analysis Dashboard

This notebook provides comprehensive performance analysis for QuantETF backtest results.

**Analysis Sections:**
1. Equity curve with dual-axis drawdown overlay
2. Monthly/yearly returns heatmap
3. Rolling Sharpe ratio (252-day window)
4. Drawdown waterfall chart
5. Returns distribution histogram
6. Underwater plot (time below high-water mark)
7. Holdings evolution over time
8. Turnover analysis

## Setup and Imports

In [ ]:
# Install required packages if needed
%pip install -q matplotlib seaborn pandas numpy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
pd.set_option('display.precision', 4)

print("Setup complete!")

## Load Backtest Data

Load the most recent backtest results from the artifacts directory.

In [ ]:
# Find the most recent backtest directory
backtests_dir = Path('../artifacts/backtests')
backtest_dirs = sorted([d for d in backtests_dir.iterdir() if d.is_dir()], reverse=True)

# Find first directory with valid data
backtest_dir = None
for d in backtest_dirs:
    metrics_file = d / 'metrics.json'
    if metrics_file.exists():
        with open(metrics_file, 'r') as f:
            metrics_test = json.load(f)
        # Check if backtest has valid results (not -100% return)
        if metrics_test.get('total_return', -1) > -0.99:
            backtest_dir = d
            break

if backtest_dir is None:
    raise FileNotFoundError("No valid backtest results found. Please run a backtest first.")

print(f"Loading backtest: {backtest_dir.name}")

# Load equity curve
equity_df = pd.read_csv(backtest_dir / 'equity_curve.csv', index_col=0, parse_dates=True)

# Load metrics
with open(backtest_dir / 'metrics.json', 'r') as f:
    metrics = json.load(f)

# Load holdings history
holdings_df = pd.read_csv(backtest_dir / 'holdings_history.csv', index_col=0, parse_dates=True)

# Load weights history
weights_df = pd.read_csv(backtest_dir / 'weights_history.csv', index_col=0, parse_dates=True)

# Load config
with open(backtest_dir / 'config.json', 'r') as f:
    config = json.load(f)

print(f"\nBacktest Period: {equity_df.index.min().date()} to {equity_df.index.max().date()}")
print(f"Total Return: {metrics['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {metrics['max_drawdown']:.2%}")
print(f"Final NAV: ${metrics['final_nav']:,.2f}")

## 1. Equity Curve with Dual-Axis Drawdown Overlay

Shows the portfolio value over time with drawdowns displayed on a secondary axis.

In [ ]:
# Calculate cumulative returns and drawdown
equity_curve = equity_df['nav']
cumulative_return = (equity_curve / equity_curve.iloc[0]) - 1

# Calculate drawdown
running_max = equity_curve.expanding().max()
drawdown = (equity_curve / running_max) - 1

# Create figure with two y-axes
fig, ax1 = plt.subplots(figsize=(14, 8))

# Plot equity curve
color = 'tab:blue'
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel('Portfolio Value ($)', color=color, fontsize=12)
ax1.plot(equity_curve.index, equity_curve.values, color=color, linewidth=2, label='Portfolio Value')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True, alpha=0.3)

# Create second y-axis for drawdown
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Drawdown (%)', color=color, fontsize=12)
ax2.fill_between(drawdown.index, drawdown.values * 100, 0, 
                 color=color, alpha=0.3, label='Drawdown')
ax2.tick_params(axis='y', labelcolor=color)

# Set title
plt.title('Equity Curve and Drawdown Analysis', fontsize=14, fontweight='bold', pad=20)

# Add legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

fig.tight_layout()
plt.show()

print(f"Peak Portfolio Value: ${equity_curve.max():,.2f}")
print(f"Maximum Drawdown: {drawdown.min():.2%}")
print(f"Current Value: ${equity_curve.iloc[-1]:,.2f}")

## 2. Monthly/Yearly Returns Heatmap

Displays returns in a calendar heatmap format for easy pattern identification.

In [ ]:
# Calculate monthly returns
daily_returns = equity_df['returns'].fillna(0)
monthly_returns = equity_df.resample('M')['nav'].last().pct_change()

# Create pivot table for heatmap
monthly_returns_df = monthly_returns.to_frame('return')
monthly_returns_df['year'] = monthly_returns_df.index.year
monthly_returns_df['month'] = monthly_returns_df.index.month

# Pivot to create heatmap data
heatmap_data = monthly_returns_df.pivot(index='year', columns='month', values='return')

# Create month names for columns
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
heatmap_data.columns = [month_names[i-1] for i in heatmap_data.columns]

# Create heatmap
plt.figure(figsize=(14, 6))
sns.heatmap(heatmap_data * 100, annot=True, fmt='.1f', cmap='RdYlGn', center=0,
            cbar_kws={'label': 'Monthly Return (%)'}, linewidths=0.5,
            vmin=-10, vmax=10)  # Set symmetric color scale

plt.title('Monthly Returns Heatmap (%)', fontsize=14, fontweight='bold', pad=15)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Year', fontsize=12)
plt.tight_layout()
plt.show()

# Calculate yearly returns
yearly_returns = equity_df.resample('Y')['nav'].last().pct_change()

print("\nYearly Returns:")
for year, ret in yearly_returns.items():
    if not pd.isna(ret):
        print(f"{year.year}: {ret:.2%}")

print(f"\nBest Month: {(monthly_returns.max()):.2%}")
print(f"Worst Month: {(monthly_returns.min()):.2%}")
print(f"Win Rate (Monthly): {(monthly_returns > 0).sum() / len(monthly_returns.dropna()):.1%}")

## 3. Rolling Sharpe Ratio (252-day window)

Shows how risk-adjusted returns evolve over time.

In [ ]:
# Calculate rolling Sharpe ratio
window = 252  # 1 year
daily_returns = equity_df['returns'].fillna(0)

rolling_mean = daily_returns.rolling(window=window).mean()
rolling_std = daily_returns.rolling(window=window).std()
rolling_sharpe = (rolling_mean / rolling_std) * np.sqrt(252)

# Plot rolling Sharpe
plt.figure(figsize=(14, 6))
plt.plot(rolling_sharpe.index, rolling_sharpe.values, linewidth=2, color='darkblue', label='Rolling Sharpe (252d)')
plt.axhline(y=0, color='black', linestyle='--', alpha=0.5, label='Zero')
plt.axhline(y=1, color='green', linestyle='--', alpha=0.5, label='Sharpe = 1.0')

plt.title('Rolling Sharpe Ratio (252-Day Window)', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Sharpe Ratio', fontsize=12)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Current Rolling Sharpe: {rolling_sharpe.iloc[-1]:.2f}")
print(f"Average Rolling Sharpe: {rolling_sharpe.mean():.2f}")
print(f"Max Rolling Sharpe: {rolling_sharpe.max():.2f}")
print(f"Min Rolling Sharpe: {rolling_sharpe.min():.2f}")

## 4. Drawdown Waterfall Chart

Identifies and visualizes individual drawdown periods.

In [ ]:
# Identify drawdown periods
is_drawdown = drawdown < -0.01  # Significant drawdowns > 1%

# Find drawdown periods
drawdown_periods = []
in_drawdown = False
start_idx = None

for i, (date, in_dd) in enumerate(zip(drawdown.index, is_drawdown)):
    if not in_drawdown and in_dd:
        start_idx = i
        in_drawdown = True
    elif in_drawdown and not in_dd:
        max_dd = drawdown.iloc[start_idx:i].min()
        drawdown_periods.append({
            'start': drawdown.index[start_idx],
            'end': drawdown.index[i-1],
            'max_dd': max_dd,
            'duration': (drawdown.index[i-1] - drawdown.index[start_idx]).days
        })
        in_drawdown = False

# Handle ongoing drawdown
if in_drawdown:
    max_dd = drawdown.iloc[start_idx:].min()
    drawdown_periods.append({
        'start': drawdown.index[start_idx],
        'end': drawdown.index[-1],
        'max_dd': max_dd,
        'duration': (drawdown.index[-1] - drawdown.index[start_idx]).days
    })

# Sort by magnitude
drawdown_periods_df = pd.DataFrame(drawdown_periods).sort_values('max_dd')

# Plot waterfall chart
if len(drawdown_periods_df) > 0:
    plt.figure(figsize=(14, 6))
    
    bars = plt.bar(range(len(drawdown_periods_df)), 
                   drawdown_periods_df['max_dd'].values * 100,
                   color='crimson', alpha=0.7, edgecolor='darkred', linewidth=1.5)
    
    plt.title('Drawdown Waterfall Chart', fontsize=14, fontweight='bold')
    plt.xlabel('Drawdown Period (sorted by magnitude)', fontsize=12)
    plt.ylabel('Maximum Drawdown (%)', fontsize=12)
    plt.grid(True, alpha=0.3, axis='y')
    plt.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
    
    # Add value labels on bars
    for i, (bar, dd) in enumerate(zip(bars, drawdown_periods_df['max_dd'].values)):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height - 0.5,
                f'{dd*100:.1f}%', ha='center', va='top', fontsize=9, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nNumber of Significant Drawdowns: {len(drawdown_periods_df)}")
    print(f"Average Drawdown: {drawdown_periods_df['max_dd'].mean():.2%}")
    print(f"Average Duration: {drawdown_periods_df['duration'].mean():.0f} days")
    print(f"\nTop 3 Drawdowns:")
    print(drawdown_periods_df[['start', 'end', 'max_dd', 'duration']].head(3).to_string())
else:
    print("No significant drawdowns detected (>1%)")

## 5. Returns Distribution Histogram

Shows the distribution of daily returns with statistical measures.

In [ ]:
# Get daily returns
returns = equity_df['returns'].dropna()

# Create histogram
fig, ax = plt.subplots(figsize=(14, 6))

# Plot histogram
ax.hist(returns * 100, bins=50, alpha=0.7, color='steelblue', 
        edgecolor='black', density=True, label='Daily Returns')

# Try to add KDE if there's enough data variation
try:
    if len(returns) > 10 and returns.std() > 1e-6:
        returns.plot.kde(ax=ax, color='darkred', linewidth=2, label='KDE', bw_method=0.3)
except Exception:
    pass  # Skip KDE if it fails

# Add vertical lines for key statistics
mean_ret = returns.mean() * 100
median_ret = returns.median() * 100
ax.axvline(mean_ret, color='green', linestyle='--', linewidth=2, label=f'Mean: {mean_ret:.3f}%')
ax.axvline(median_ret, color='orange', linestyle='--', linewidth=2, label=f'Median: {median_ret:.3f}%')
ax.axvline(0, color='black', linestyle='-', linewidth=1, alpha=0.5)

plt.title('Daily Returns Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Daily Return (%)', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate statistics
try:
    from scipy import stats
    skewness = stats.skew(returns.dropna())
    kurtosis = stats.kurtosis(returns.dropna())
except ImportError:
    skewness = 0.0
    kurtosis = 0.0
    print("Note: scipy not available, skewness and kurtosis set to 0")

print("\nReturns Statistics:")
print(f"Mean Daily Return: {returns.mean():.4%}")
print(f"Median Daily Return: {returns.median():.4%}")
print(f"Std Dev (Daily): {returns.std():.4%}")
print(f"Annualized Volatility: {returns.std() * np.sqrt(252):.2%}")
if skewness != 0 or kurtosis != 0:
    print(f"Skewness: {skewness:.2f}")
    print(f"Kurtosis: {kurtosis:.2f}")
print(f"Best Day: {returns.max():.2%}")
print(f"Worst Day: {returns.min():.2%}")
print(f"Positive Days: {(returns > 0).sum()} ({(returns > 0).mean():.1%})")

## 6. Underwater Plot (Time Below High-Water Mark)

Shows how long the portfolio stays below its previous peak.

In [ ]:
# Calculate underwater (already have drawdown)
underwater = drawdown.copy()

# Create underwater plot
plt.figure(figsize=(14, 6))

# Fill area below zero
plt.fill_between(underwater.index, underwater.values * 100, 0,
                 where=(underwater < 0), color='red', alpha=0.4, 
                 label='Underwater (Below Peak)', interpolate=True)

# Add zero line
plt.axhline(y=0, color='black', linestyle='-', linewidth=1.5, label='High-Water Mark')

plt.title('Underwater Plot: Time Below High-Water Mark', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Drawdown from Peak (%)', fontsize=12)
plt.legend(loc='lower left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate underwater statistics
underwater_periods = underwater < 0
total_days = len(underwater)
underwater_days = underwater_periods.sum()
pct_underwater = underwater_days / total_days

# Find longest underwater streak
underwater_streaks = underwater_periods.groupby(
    (underwater_periods != underwater_periods.shift()).cumsum()
).sum()
longest_streak = underwater_streaks.max() if len(underwater_streaks) > 0 else 0

# Average depth while underwater
if underwater_days > 0:
    avg_underwater_depth = underwater[underwater_periods].mean()
else:
    avg_underwater_depth = 0

print("\nUnderwater Statistics:")
print(f"Total Days: {total_days}")
print(f"Days Underwater: {underwater_days} ({pct_underwater:.1%})")
print(f"Longest Underwater Streak: {longest_streak} days")
print(f"Average Underwater Depth: {avg_underwater_depth:.2%}")

## 7. Holdings Evolution Over Time

Visualizes how portfolio holdings change over the backtest period.

In [ ]:
# Use weights for cleaner visualization
weights_pct = weights_df * 100

# Create stacked area chart
fig, ax = plt.subplots(figsize=(14, 8))

# Plot stacked area
weights_pct.plot.area(ax=ax, stacked=True, alpha=0.7, linewidth=0)

plt.title('Portfolio Holdings Evolution Over Time', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Weight (%)', fontsize=12)
plt.legend(title='Ticker', bbox_to_anchor=(1.05, 1), loc='upper left', ncol=1)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

# Calculate holding statistics
avg_weights = weights_df.mean().sort_values(ascending=False)
hold_frequency = (weights_df > 0.001).mean().sort_values(ascending=False)

print("\nTop 10 Holdings by Average Weight:")
for ticker, weight in avg_weights.head(10).items():
    freq = hold_frequency[ticker]
    print(f"{ticker:6s}: {weight:6.2%} (held {freq:.1%} of time)")

print(f"\nAverage Number of Positions: {(weights_df > 0.001).sum(axis=1).mean():.1f}")

## 8. Turnover Analysis

Analyzes portfolio turnover and trading activity.

In [ ]:
# Calculate turnover (sum of absolute weight changes / 2)
weight_changes = weights_df.diff().abs()
turnover = weight_changes.sum(axis=1) / 2

# Create subplot for turnover metrics
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))

# 1. Turnover over time
ax1.plot(turnover.index, turnover.values * 100, linewidth=2, color='steelblue', marker='o', markersize=4)
ax1.set_title('Portfolio Turnover Over Time', fontweight='bold')
ax1.set_ylabel('Turnover (%)', fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.axhline(turnover.mean() * 100, color='red', linestyle='--', 
            label=f'Average: {turnover.mean():.2%}', alpha=0.7)
ax1.legend()

# 2. Portfolio concentration (HHI)
hhi = (weights_df ** 2).sum(axis=1)
effective_n = 1 / hhi

ax2.plot(hhi.index, hhi.values, linewidth=2, color='darkgreen', marker='o', markersize=4)
ax2.set_title('Portfolio Concentration (HHI)', fontweight='bold')
ax2.set_ylabel('Herfindahl Index', fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.axhline(hhi.mean(), color='red', linestyle='--', 
            label=f'Average: {hhi.mean():.3f}', alpha=0.7)
ax2.legend()

# 3. Number of positions
num_positions = (weights_df > 0.001).sum(axis=1)
ax3.plot(num_positions.index, num_positions.values, linewidth=2, 
         color='darkorange', marker='o', markersize=4)
ax3.set_title('Number of Positions Held', fontweight='bold')
ax3.set_ylabel('Count', fontsize=10)
ax3.set_xlabel('Date', fontsize=10)
ax3.grid(True, alpha=0.3)
ax3.axhline(num_positions.mean(), color='red', linestyle='--', 
            label=f'Average: {num_positions.mean():.1f}', alpha=0.7)
ax3.legend()

# 4. Trade size distribution
all_trades = weight_changes.values.flatten()
all_trades = all_trades[all_trades > 0.001]  # Filter out tiny changes

ax4.hist(all_trades * 100, bins=30, color='crimson', alpha=0.7, edgecolor='black')
ax4.set_title('Trade Size Distribution', fontweight='bold')
ax4.set_xlabel('Trade Size (% of Portfolio)', fontsize=10)
ax4.set_ylabel('Frequency', fontsize=10)
ax4.grid(True, alpha=0.3, axis='y')
ax4.axvline(np.median(all_trades) * 100, color='blue', linestyle='--',
            label=f'Median: {np.median(all_trades):.2%}', linewidth=2)
ax4.legend()

plt.tight_layout()
plt.show()

# Summary statistics
print("\nTurnover Statistics:")
print(f"Average Turnover per Rebalance: {turnover.mean():.2%}")
print(f"Total Turnover: {turnover.sum():.2%}")
print(f"Annualized Turnover: {turnover.sum() / (len(equity_df) / 252):.2%}")
print(f"Max Single Rebalance Turnover: {turnover.max():.2%}")

print("\nConcentration Statistics:")
print(f"Average HHI: {hhi.mean():.3f}")
print(f"Average Effective N: {effective_n.mean():.2f}")
print(f"Average Positions: {num_positions.mean():.1f}")

print("\nTrade Statistics:")
print(f"Total Trades: {len(all_trades)}")
print(f"Median Trade Size: {np.median(all_trades):.2%}")
print(f"Average Trade Size: {np.mean(all_trades):.2%}")
print(f"Largest Trade: {np.max(all_trades):.2%}")

## Summary

This analysis provides a comprehensive view of the backtest performance including:

- **Equity Performance**: Overall returns and drawdown patterns
- **Risk Metrics**: Sharpe ratio evolution and return distributions
- **Portfolio Characteristics**: Holdings, turnover, and concentration
- **Underwater Analysis**: Time spent below previous highs

The visualizations help identify strengths, weaknesses, and areas for strategy improvement.